In [2]:
import pyodbc
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import pandas as pd

class SQLConnection:
    def __init__(self,sql_path, db_server=os.getenv("DB_SERVER"), db_name=os.getenv("DB_NAME"), db_driver=os.getenv("DB_DRIVER"),params={"medico": "PSICOLOGÍA","fechaini": "20230101","fechafin": "20250504"}):
        load_dotenv()
        self.db_name = db_name
        self.db_server = db_server
        self.db_driver = db_driver
        self.params = params
        self.sql_path = sql_path
        
    def get_connection_string(self):
        db_connection_string = (f"mssql+pyodbc://@{self.db_server}/{self.db_name}"f"?driver={self.db_driver.replace(' ', '+')}&trusted_connection=yes")
        return db_connection_string
    
    def create_engine_connection(self):
        connection_string = self.get_connection_string()
        engine = create_engine(connection_string)
        return engine
    
    def generate_dataframe(self, nregistros=10000):
        self.nregistros = nregistros
        with open(self.sql_path, "r", encoding="utf-8") as file: 
            query = file.read()
        
        engine = self.create_engine_connection()
        with engine.connect() as connection:
            df = pd.read_sql(text(query), connection, params=self.params)
        return df.head(self.nregistros)
    
    

        

In [3]:
params={"medico": "PSICOLOGÍA","fechaini": "20230101","fechafin": "20250504"}
sql_path=os.path.join("..", "sql_queries", "queries.sql")
sqlconection=SQLConnection(sql_path=sql_path, params=params)
historias_clinicas_df = sqlconection.generate_dataframe()

In [4]:
historias_clinicas_df.head()

,SEXO,EDAD,GRUPO,ESPECIALIDAD_MEDICA,SUBJETIVO,OBJETIVO,Concatenada
0,Masculino,10,T. Externalizantes,PSICOLOGÍA,None,None,
1,Femenino,38,Otros Trastornos,PSICOLOGÍA,"Paciente refiere: ""Me empezaron a dar como uno...","Paciente alerta, colaboradora con apariencia o...","Paciente refiere: ""Me empezaron a dar como uno..."
2,Masculino,22,T. Externalizantes,PSICOLOGÍA,"Paciente refiere ""Me he sentido muy mal, en el...","Paciente a quien evaluó por primera vez, alert...","Paciente refiere ""Me he sentido muy mal, en el..."
3,Masculino,9,Otros Trastornos,PSICOLOGÍA,"La madre refiere ""el viene por un acompañamien...",None,"La madre refiere ""el viene por un acompañamien..."
4,Masculino,28,Otros Trastornos,PSICOLOGÍA,"Paciente refiere ""Estas cosas que han pasado m...","Paciente quien evalúo por primera vez, alerta,...","Paciente refiere ""Estas cosas que han pasado m..."


In [5]:
# etl.py

class EDAdataset():
    
    def __init__(self, df):
        # Crea una copia del dataframe original para evitar modificarlo directamente
        self.df = df.copy()
    
    def lowercase_columns(self):
        # Convierte los nombres de las columnas a minúsculas
        columnames = self.df.columns.str.lower()
        self.df.columns = columnames  # Convert columns to lowercase
    
    def remove_nulls(self):
        # Eliminar filas donde ambas columnas "subjetivo" y "objetivo" son nulas
        self.df = self.df.dropna(subset=["subjetivo", "objetivo"], how="all") # Eliminar filas donde ambas columnas son nulas 
        return self.df

    def capitalize_grupos(self, column_name="grupo"):
        # Capitaliza la primera letra de cada palabra en la columna "grupo"
        self.df[column_name] = self.df[column_name].str.capitalize()
        return self.df
    
    def dataset_eda(self, df):
        # Realiza todas las transformaciones de EDA en el dataframe
        self.df = df
        self.lowercase_columns()
        self.df = self.remove_nulls()
        self.df = self.capitalize_grupos()
        return self.df

In [6]:
df_copy = EDAdataset(historias_clinicas_df)
df_final = df_copy.dataset_eda(historias_clinicas_df)

C:\Users\ISAIAS_QUINTERO\AppData\Local\Temp\ipykernel_25516\3174529181.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[column_name] = self.df[column_name].str.capitalize()


In [7]:
df_final.head()

,sexo,edad,grupo,especialidad_medica,subjetivo,objetivo,concatenada
1,Femenino,38,Otros trastornos,PSICOLOGÍA,"Paciente refiere: ""Me empezaron a dar como uno...","Paciente alerta, colaboradora con apariencia o...","Paciente refiere: ""Me empezaron a dar como uno..."
2,Masculino,22,T. externalizantes,PSICOLOGÍA,"Paciente refiere ""Me he sentido muy mal, en el...","Paciente a quien evaluó por primera vez, alert...","Paciente refiere ""Me he sentido muy mal, en el..."
3,Masculino,9,Otros trastornos,PSICOLOGÍA,"La madre refiere ""el viene por un acompañamien...",None,"La madre refiere ""el viene por un acompañamien..."
4,Masculino,28,Otros trastornos,PSICOLOGÍA,"Paciente refiere ""Estas cosas que han pasado m...","Paciente quien evalúo por primera vez, alerta,...","Paciente refiere ""Estas cosas que han pasado m..."
5,Femenino,8,Otros trastornos,PSICOLOGÍA,"La madre refiere ""Ella ha manifestado ciertas ...","Paciente ingresa en compañía de la madre, aler...","La madre refiere ""Ella ha manifestado ciertas ..."


In [8]:
df_final.tail()

,sexo,edad,grupo,especialidad_medica,subjetivo,objetivo,concatenada
9995,Masculino,17,T. personalidad,PSICOLOGÍA,"""Fumo todos los días, como 3 baretos al día, p...","Paciente alerta, colaborador, orientado en sus...","""Fumo todos los días, como 3 baretos al día, p..."
9996,Femenino,42,T. de ansiedad,PSICOLOGÍA,"SEGUIMIENTO: La paciente refiere ""ya desde psi...","Paciente quien evalúo por primera vez, con bas...","SEGUIMIENTO: La paciente refiere ""ya desde psi..."
9997,Femenino,40,T. de ansiedad,PSICOLOGÍA,Paciente refiere que viene con antelación emoc...,"Paciente con síntomas de ansiedad y depresión,...",Paciente refiere que viene con antelación emoc...
9998,Femenino,37,T. de ansiedad,PSICOLOGÍA,"SEGUIMIENTO: La paciente refiere ""me he sentid...","Paciente quien evalúo por primera vez, con bas...","SEGUIMIENTO: La paciente refiere ""me he sentid..."
9999,Femenino,24,T. depresivos,PSICOLOGÍA,"SEGUIMIENTO: La paciente refiere ""me he sentid...",Paciente atendida en modalidad de teleconsulta...,"SEGUIMIENTO: La paciente refiere ""me he sentid..."


In [17]:
# Feature_Engineer.py

import pandas as pd
import re
import spacy
from sklearn import preprocessing

class PreprocesadorTexto:
    def __init__(self, df, stopwords={
            "medico", "paciente", "psicologo", "psicologa",
            "psicologia", "psicoterapeuta", "psicoterapia", "refiere"}):
        self.df = df.copy()
        self.stopwords = stopwords
        self.nlp = spacy.load("es_core_news_lg")
        self.stopwords = self.nlp.Defaults.stop_words.union(self.stopwords)

    def concatenar_columnas(self, df, columna1="subjetivo", columna2="objetivo", nueva_columna="concatenada"):
        self.df = df
        self.df[nueva_columna] = self.df[columna1].astype(str) + " " + self.df[columna2].astype(str)
        return self.df

    def expresiones_regulares(self, columna):
        self.df[columna] = (
            self.df[columna]
            .astype(str)
            .str.lower()
            .apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'[^a-zñü ]', '', x)).strip())
        )

    def tokenizar(self, columna):
        self.df[columna] = (
            self.df[columna]
            .astype(str)
            .fillna("")
            .apply(lambda x: [
                token.text for token in self.nlp(x)
                if token.text.lower() not in self.stopwords and not token.is_punct and not token.is_space
            ])
        )

    def lematizar(self, columna):
        self.df[columna] = self.df[columna].apply(
            lambda x: [token.lemma_ for token in self.nlp(" ".join(x))] if isinstance(x, list) else []
        )

    def label_encodering(self, columna, nueva_columna, tipo="sexo"):
        label_encoder = preprocessing.LabelEncoder()
        self.df[nueva_columna] = label_encoder.fit_transform(self.df[columna])
        mapping_df = pd.DataFrame({
            tipo.capitalize(): label_encoder.classes_,
            'Codigo': label_encoder.transform(label_encoder.classes_)
        })
        return mapping_df

    def procesar(self, columna_texto, columna_sexo=None, columna_grupo=None):
        self.expresiones_regulares(columna_texto)
        self.tokenizar(columna_texto)
        self.lematizar(columna_texto)

        mappings = {}
        if columna_sexo:
            mappings["sexo"] = self.label_encodering(columna_sexo, "sexo_codificado", tipo="sexo")
        if columna_grupo:
            mappings["grupo"] = self.label_encodering(columna_grupo, "grupo_codificado", tipo="grupo")
        return self.df, mappings


In [11]:
stopwords={
            "medico", "paciente", "psicologo", "psicologa",
            "psicologia", "psicoterapeuta", "psicoterapia", "refiere"
        }

preprocesador = PreprocesadorTexto(df_final, stopwords=stopwords)
df_procesado, mapeos = preprocesador.procesar(
    columna_texto="concatenada", # Columna que contiene el texto a procesar
    columna_sexo="sexo", # Columna que contiene la variable sexo
    columna_grupo="grupo" # Columna que contiene la variable grupo
)

In [12]:
df_procesado.head()

,sexo,edad,grupo,especialidad_medica,subjetivo,objetivo,concatenada,sexo_codificado,grupo_codificado
1,Femenino,38,Otros trastornos,PSICOLOGÍA,"Paciente refiere: ""Me empezaron a dar como uno...","Paciente alerta, colaboradora con apariencia o...","[empezar, episodio, tom, pastilla, dormir, ten...",0,0
2,Masculino,22,T. externalizantes,PSICOLOGÍA,"Paciente refiere ""Me he sentido muy mal, en el...","Paciente a quien evaluó por primera vez, alert...","[sentido, trabajar, concentrar yo, cosa, vidat...",1,4
3,Masculino,9,Otros trastornos,PSICOLOGÍA,"La madre refiere ""el viene por un acompañamien...",None,"[madre, venir, acompañamiento, emocional, sepr...",1,0
4,Masculino,28,Otros trastornos,PSICOLOGÍA,"Paciente refiere ""Estas cosas que han pasado m...","Paciente quien evalúo por primera vez, alerta,...","[cosa, problema, empresa, ocasión, saludar, sa...",1,0
5,Femenino,8,Otros trastornos,PSICOLOGÍA,"La madre refiere ""Ella ha manifestado ciertas ...","Paciente ingresa en compañía de la madre, aler...","[madre, manifestar, conducta, palabrasello, re...",0,0


In [13]:
mapeos

{'sexo':         Sexo  Codigo
 0   Femenino       0
 1  Masculino       1,
 'grupo':                 Grupo  Codigo
 0    Otros trastornos       0
 1    T. de adaptación       1
 2      T. de ansiedad       2
 3       T. depresivos       3
 4  T. externalizantes       4
 5     T. personalidad       5}

In [18]:
# train.py

# Convertir a excel
df_procesado.to_excel("historias_clinicas_procesadas.xlsx", index=False)